In [ ]:
GITHUB_USERNAME = "$GITHUB_USERNAME$"
GITHUB_REF = "$GITHUB_REF$"
NOTEBOOK_TYPE = "$NOTEBOOK_TYPE$"
PYTHON_VERSION = "$PYTHON_VERSION$"
IPYTHON_VERSION = "$IPYTHON_VERSION$"

In [ ]:
import warnings
from pathlib import Path

import requests


warnings.filterwarnings('error', module='davos')

if NOTEBOOK_TYPE == 'colab':
    # utils module doesn't exist on colab VM, so get current version from GitHub
    utils_module = Path('utils.py').resolve()
    response = requests.get(f'https://raw.githubusercontent.com/{GITHUB_USERNAME}/davos/{GITHUB_REF}/tests/utils.py')
    utils_module.write_text(response.text)
    # also need to install davos locally
    from utils import install_davos
    install_davos(source='github', ref=GITHUB_REF, fork=GITHUB_USERNAME)

In [ ]:
import inspect
import json
import subprocess
import sys

if sys.version_info < (3, 8):
    import importlib_metadata as metadata
else:
    from importlib import metadata

import davos
import IPython
from packaging.specifiers import SpecifierSet

from utils import (
    is_imported, 
    is_installed, 
    mark, 
    raises, 
    run_tests, 
    TestingEnvironmentError
)

In [ ]:
IPYTHON_SHELL = get_ipython()

# tests for general testing environment & package initialization
tests GitHub runner itself, as well as contents of `__init__.py` & `implementations.__init__.py`

In [ ]:
def test_import_davos():
    global davos
    import davos
    assert is_imported('davos')

In [ ]:
def test_expected_python_version():
    installed_version = '.'.join(map(str, sys.version_info[:2]))
    expected_version = PYTHON_VERSION
    if installed_version != expected_version:
        raise TestingEnvironmentError(
            f"Test environment has Python {sys.version.split()[0]}, expected "
            "{PYTHON_VERSION}"
        )

In [ ]:
@mark.jupyter
def test_notebook_using_kernel_python():
    if not sys.executable.endswith('envs/kernel-env/bin/python'):
        raise TestingEnvironmentError(
            "Notebook does not appear to be using the correct python "
            "executable. Expected a path ending in "
            f"'envs/kernel-env/bin/python', found {sys.executable}"
        )

In [ ]:
@mark.skipif(IPYTHON_VERSION == 'latest', reason="runs when IPYTHON_VERSION != 'latest'")
def test_expected_ipython_version():
    ipy_version = metadata.version('IPython')
    if ipy_version not in SpecifierSet(f'=={IPYTHON_VERSION}'):
        raise TestingEnvironmentError(
            f"Test environment has IPython=={IPython.__version__}, expected "
            f"{IPYTHON_VERSION}"
        )

In [ ]:
@mark.skipif(IPYTHON_VERSION != 'latest', reason="runs when IPYTHON_VERSION == 'latest'")
def test_latest_ipython_version():
    pip_exe = davos.config.pip_executable
    outdated_pkgs = subprocess.check_output(
            [pip_exe, 'list', '--outdated', '--format', 'json'], encoding='utf-8'
        )
    outdated_pkgs_json = json.loads(outdated_pkgs)
    for pkg in outdated_pkgs_json:
        if pkg['name'] == 'ipython':
            raise TestingEnvironmentError(
                f"Test environment has IPython=={pkg['version']}, expected "
                f"latest version (IPython=={pkg['latest_version']})"
            )

In [ ]:
def test_scipy_installed():
    """used as an example package for some tests"""
    assert is_installed('scipy')

In [ ]:
def test_fastdtw_installed():
    """used as an example package for some tests"""
    assert is_installed('fastdtw==0.3.4')

In [ ]:
def test_tqdm_installed():
    """used as an example package for some tests"""
    assert is_installed('tqdm')
    import tqdm
    assert tqdm.__version__ != '==4.45.0'

In [ ]:
def test_smuggle_in_namespace():
    assert 'smuggle' in globals()
    assert 'smuggle' in IPYTHON_SHELL.user_ns
    assert globals()['smuggle'] is IPYTHON_SHELL.user_ns['smuggle']

In [ ]:
def test_activated_on_import():
    assert davos.config.active is True

In [ ]:
def test_deactivate_reactivate_toplevel():
    assert davos.active is True
    
    davos.active = False
    assert not davos.active 
    
    with raises(NameError, match="name 'smuggle' is not defined"):
        smuggle ast
        
    davos.active = True
    assert davos.active

In [ ]:
def test_config_properties_accessible_toplevel():
    """
    test that davos.config fields are accesible from the top-level davos 
    namespace
    """
    config_fields = ['active', 'auto_rerun', 'confirm_install', 
                     'noninteractive', 'pip_executable', 'project', 
                     'suppress_stdout', 'environment', 'ipython_shell', 
                     'smuggled']
    failed = []
    for field in config_fields:
        # values should not only be equal, they should be references to 
        # the *same object*
        if getattr(davos, field) is not getattr(davos.config, field):
            failed.append(field)
    
    assert not failed, (
        "The following fields were not the same when accessed via "
        f"davos.config and the top-level davos module: {', '.join(failed)}"
    )

In [ ]:
def test_set_new_attrs_toplevel_only():
    """
    Setting an attribute on the top-level davos module that is *not* 
    already defined by davos.config should affect only the top-level 
    module.
    """
    assert not hasattr(davos, 'undefined_attr')
    
    davos.undefined_attr = 'test-value'
    assert hasattr(davos, 'undefined_attr')
    assert davos.undefined_attr == 'test-value'
    assert not hasattr(davos.config, 'undefined_attr')
    
    del davos.undefined_attr

In [ ]:
def test_all_configurable_fields_settable_via_configure():
    all_properties = []
    for name, val in davos.core.config.DavosConfig.__dict__.items():
        if isinstance(val, property):
            all_properties.append(name)
    read_only_fields = {'environment', 'ipython_shell', 'smuggled'}
    not_implemented_fields = {'conda_avail', 'conda_env', 'conda_envs_dirs'}
    configurable_fields = (set(all_properties) 
                           - read_only_fields 
                           - not_implemented_fields)
    configure_func_kwargs = set(inspect.signature(davos.configure).parameters)
    assert not configurable_fields.symmetric_difference(
        configure_func_kwargs
    ), (
        f"configurable fields: {configurable_fields}\ndavos.configure kwargs: "
        f"{configure_func_kwargs}"
    )

In [ ]:
def test_confirm_install_noninteractive_true_fails():
    """
    davos.configure function should disallow passing both 
    `confirm_install=True` and `noninteractive=True`.
    """
    with raises(davos.core.exceptions.DavosConfigError):
        davos.configure(confirm_install=True, noninteractive=True)

In [ ]:
def test_swap_confirm_install_noninteractive_succeeds():
    """
    Simultaneously disabling non-interactive mode and enabling the 
    confirm_install option should succeed. i.e., when 
    `davos.noninteractive` is initially `True` and both 
    `noninteractive=False` and `confirm_install=True` are passed, 
    `config.noninteractive` should be set to `False` before
    `config.confirm_install` is set to `True` to ensure both are not 
    `True` at the same time.
    """
    # record initial values to restore later
    initial_confirm_install_value = davos.confirm_install
    initial_noninteractive_value = davos.noninteractive
    
    try:
        # set up initial conditions
        davos.confirm_install = False
        davos.noninteractive = True
        
        davos.configure(confirm_install=True, noninteractive=False)
        
        # now reset initial conditions and test passing the arguments 
        # in the opposite order
        davos.confirm_install = False
        davos.noninteractive = True
        
        davos.configure(noninteractive=False, confirm_install=True)
    finally:
        davos.config._confirm_install = initial_confirm_install_value
        davos.config._noninteractive = initial_noninteractive_value

In [ ]:
def test_configure_resets_fields_on_fail():
    active_before = davos.config.active
    confirm_install_before = davos.config.confirm_install
    with raises(davos.core.exceptions.DavosConfigError):
        davos.configure(
            active=False, 
            confirm_install=True, 
            suppress_stdout='BAD VALUE'
        )
    assert davos.config.active is active_before
    assert davos.config.confirm_install is confirm_install_before

In [ ]:
run_tests()